In [1]:
# init vader
import nltk

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/fagh/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# init hastags, map to lower

tag_list = ['DeepstateVirus', 'DeepStateVaccine', 'DeepStateFauci', 'QAnon', 'Agenda21', 'CCPVirus',
            'ClimateChangeHoax', 'GlobalWarmingHoax', 'ChinaLiedPeopleDied', 'SorosVirus', '5GCoronavirus', 'MAGA',
            'WWG1WGA', 'Chemtrails', 'flatEarth', 'MoonLandingHoax', 'moonhoax', 'illuminati', 'pizzaGateIsReal',
            'PedoGateIsReal', '911truth', '911insidejob', 'reptilians']

tag_list_lower = list(map(lambda x: x.lower(), tag_list))

In [3]:
# func to exec basic select
from psycopg2.extras import RealDictCursor
import psycopg2

""" Connect to the PostgreSQL database server """
conn = None


def exec_sql(sql: str):
    try:
        conn = psycopg2.connect("dbname=data user=postgres password=root host=172.23.0.2")
        cur = conn.cursor(cursor_factory=RealDictCursor)

        cur.execute(sql)
        result = cur.fetchall()
        cur.close()
        return result
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [4]:
# get all ids of hoax hashtags
tag_list_ids = []
for hoax in tag_list_lower:
    sql = f'select * from hashtags where LOWER(value) like \'%{hoax}%\''
    rows = exec_sql(sql)
    tag_list_ids.extend(map(lambda x: x['id'], rows))


In [5]:
# get tweet ids which need to be processed with vader

sql = f'select DISTINCT tweet_hashtags.tweet_id from  tweet_hashtags inner join tweets t on tweet_hashtags.tweet_id = t.id where tweet_hashtags.hashtag_id in {str(tuple(tag_list_ids))} and t.neg is null'
rows = exec_sql(sql)

ids_to_process = list(map(lambda x: x['tweet_id'], rows))



In [6]:
# func to sanitize content
import re

def filter_content(content):
    content = re.sub(r'\s?[@#]\w*', '', content)
    content = re.sub(r'\s?https:\/\/t\.co\/\w*', '', content)
    emoj = re.compile("\s?["
                      u"\U0001F600-\U0001F64F"  # emoticons
                      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                      u"\U0001F680-\U0001F6FF"  # transport & map symbols
                      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                      u"\U00002500-\U00002BEF"  # chinese char
                      u"\U00002702-\U000027B0"
                      u"\U00002702-\U000027B0"
                      u"\U000024C2-\U0001F251"
                      u"\U0001f926-\U0001f937"
                      u"\U00010000-\U0010ffff"
                      u"\u2640-\u2642"
                      u"\u2600-\u2B55"
                      u"\u200d"
                      u"\u23cf"
                      u"\u23e9"
                      u"\u231a"
                      u"\ufe0f"  # dingbats
                      u"\u3030"
                      "]+", re.UNICODE)
    content = emoj.sub(r'', content)
    return content


In [12]:
# func to parallel calc and update vader data from tweets content
threads = 6
from multiprocessing import Pool

def update_vader(processing_batch):
    conn = psycopg2.connect("dbname=data user=postgres password=root host=172.23.0.2")
    cur = conn.cursor(cursor_factory=RealDictCursor)
    for processing_id in processing_batch:
        sql = f'select content from tweets where id = \'{processing_id}\''
        cur.execute(sql)
        rows = cur.fetchall()

        content = filter_content(rows[0]['content'])
        vader = sid.polarity_scores(content)
        sql = 'UPDATE tweets SET neg = %s, pos = %s, neu = %s, compound = %s WHERE id = %s'
        data = [vader["neg"], vader["pos"], vader["neu"], vader["compound"], processing_id]
        cur.execute(sql, data)
        conn.commit()
    conn.close()


if ids_to_process:
    import numpy
    batches = numpy.array_split(ids_to_process, threads)
    with Pool(processes=threads) as pool:
        for i in pool.imap_unordered(update_vader, batches):
            pass
print("danone")




danone
